In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
#from config import g_key
from citipy import citipy

In [2]:
#set "cit center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)


In [3]:
#turn cities into df
df_city = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [4]:
df_city.head(20)

,City,lat,lng
0,copperas cove,31.207138,-98.338127
1,uvalde,29.097840,-100.030438
2,bryan,30.685860,-96.441758
3,beeville,28.832041,-97.975450
4,victoria,29.096786,-97.310989
5,lufkin,30.921671,-94.354205
6,brenham,30.069881,-96.883497
7,orange,30.480449,-93.714422
8,temple,31.302473,-97.353020
9,san marcos,29.966729,-97.977004


In [5]:
#check length of cities
len(list(df_city["City"].unique()))

114

In [6]:
cities = list(df_city["City"].unique())
target_cities=[]

for i in cities:
    c=str(i).replace(" ","+")
    target_cities.append(c)

target_cities

['copperas+cove',
 'uvalde',
 'bryan',
 'beeville',
 'victoria',
 'lufkin',
 'brenham',
 'orange',
 'temple',
 'san+marcos',
 'port+arthur',
 'jacksonville',
 'nederland',
 'nacogdoches',
 'brownwood',
 'austin',
 'gatesville',
 'conroe',
 'huntsville',
 'corsicana',
 'kerrville',
 'baytown',
 'palestine',
 'fredericksburg',
 'liberty',
 'el+campo',
 'leon+valley',
 'college+station',
 'galveston',
 'san+angelo',
 'wharton',
 'hondo',
 'lumberton',
 'lockhart',
 'belton',
 'georgetown',
 'pleasanton',
 'lakeway',
 'texas+city',
 'south+houston',
 'natchitoches',
 'robinson',
 'freeport',
 'woodway',
 'alvin',
 'sulphur',
 'angleton',
 'atascocita',
 'seguin',
 'new+braunfels',
 'santa+fe',
 'taylor',
 'la+marque',
 'mission+bend',
 'canyon+lake',
 'tomball',
 'del+rio',
 'hewitt',
 'rosenberg',
 'missouri+city',
 'anderson+mill',
 'bay+city',
 'west+university+place',
 'kyle',
 'pflugerville',
 'bellmead',
 'live+oak',
 'leander',
 'bellaire',
 'san+antonio',
 'aldine',
 'katy',
 'kirb

In [7]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

 base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
temp=[]
wind_speed=[]
pressure=[]
city_name=[]

for city in target_cities[:4]:
    query_url=f"{base_current_url}appid={weather_api_key}&q={city},US&units=imperial"
    req = requests.get(query_url)
    response1=req.json()
    latitude=response1["coord"]["lat"]
    longitude=response1["coord"]["lon"]

    for x in unix_time():
        historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
        hist_json = requests.get(historical_url).json()
        temp.append(hist_json["current"]["temp"])
        pressure.append(hist_json["current"]["pressure"])
        wind_speed.append(hist_json["current"]["wind_speed"])
        city_name.append(response1['name'])
        print(historical_url)

In [48]:
#set base url and define function to find OpenWeather cities' based on city name from nearest place API and feed back into
#OpenWeather one call API to ensure proper lat and lng

def find_location(target_cities):
    unix_time()
    base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
    temp=[]
    wind_speed=[]
    pressure=[]
    hour_temp =[]
    hour_pressure =[]
    hour_wind_speed =[]
    city_name=[]
    country_code=[]
    for city in target_cities[:5]:
        try: 
            query_url=f"{base_current_url}appid={weather_api_key}&q={city},Texas,USA&units=imperial"
            req = requests.get(query_url)
            response1=req.json()
            latitude=response1["coord"]["lat"]
            longitude=response1["coord"]["lon"]
#             print(response1)

        except:
            print("City not found. Skipping...")
            pass
        if response1["sys"]["country"]=="US":
            for x in unix_time():
                historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
                hist_json = requests.get(historical_url).json()
                for i in range(len(hist_json["hourly"][0])):
                    hour_temp.append(hist_json["hourly"][0]["temp"])
                    hour_pressure.append(hist_json["hourly"][0]["pressure"])
                    hour_wind_speed.append(hist_json["hourly"][0]["wind_speed"])
                    #city_name.append(response1['name'])
                    #country_code.append(response1["sys"]["country"])
                
    d=dict()
    #d["City Name"] = city_name
    #d["Country"] = country_code
    d["Temperature"] = hour_temp
    d["Wind_Speed"] = hour_wind_speed
    d["Pressure"] = hour_pressure    
    return(d)
        #print ("--"*30)

In [83]:
unix_time()

[1630386000, 1630299600, 1630213200, 1630126800, 1630040400]

In [84]:
for i, x in df_city.head(30).iterrows():
    lat = x["lat"]
    lng = x["lng"]
    City = x["City"]
    historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={lng}&dt=1630386000&appid={weather_api_key}&units=imperial"
    hist_json = requests.get(historical_url).json()

    print(City, len(hist_json["hourly"]))

copperas cove 24
uvalde 24
bryan 24
beeville 24
victoria 24


In [75]:
historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.207138&lon=-98.338127&dt=1630461738&appid={weather_api_key}&units=imperial"
historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.369003&lon=-97.767841&dt=1630461738&appid={weather_api_key}&units=imperial"
hist_json = requests.get(historical_url).json()

hour_temp =[]
hour_pressure =[]
hour_wind_speed =[]
for i in range(len(hist_json["hourly"][0])):
    hour_temp.append(hist_json["hourly"][0]["temp"])
    hour_pressure.append(hist_json["hourly"][0]["pressure"])
    hour_wind_speed.append(hist_json["hourly"][0]["wind_speed"])
print(hour_temp)
print((hour_pressure))
print((hour_wind_speed))
print(len(hist_json["hourly"][0]))


[93.9, 93.9, 93.9, 93.9, 93.9, 93.9, 93.9, 93.9, 93.9, 93.9, 93.9, 93.9]
[1005, 1005, 1005, 1005, 1005, 1005, 1005, 1005, 1005, 1005, 1005, 1005]
[5.75, 5.75, 5.75, 5.75, 5.75, 5.75, 5.75, 5.75, 5.75, 5.75, 5.75, 5.75]
12


In [81]:
print(len(hist_json["hourly"]))

12


In [72]:
import json
print(json.dumps(hist_json, indent=4))

{
    "lat": 31.2071,
    "lon": -98.3381,
    "timezone": "America/Chicago",
    "timezone_offset": -18000,
    "current": {
        "dt": 1630461738,
        "sunrise": 1630411748,
        "sunset": 1630457906,
        "temp": 85.28,
        "feels_like": 90.09,
        "pressure": 1008,
        "humidity": 61,
        "dew_point": 70.3,
        "uvi": 0,
        "clouds": 40,
        "visibility": 10000,
        "wind_speed": 1.99,
        "wind_deg": 141,
        "wind_gust": 7,
        "weather": [
            {
                "id": 802,
                "main": "Clouds",
                "description": "scattered clouds",
                "icon": "03n"
            }
        ]
    },
    "hourly": [
        {
            "dt": 1630454400,
            "temp": 91.72,
            "feels_like": 96.94,
            "pressure": 1007,
            "humidity": 48,
            "dew_point": 69.24,
            "uvi": 0.31,
            "clouds": 40,
            "visibility": 10000,
            "w

In [64]:
df2 =pd.DataFrame(find_location(target_cities))
df2

,City Name,Country,Temperature,Wind_Speed,Pressure
0,Copperas Cove,US,80.10,0.00,1010
1,Copperas Cove,US,77.02,0.00,1013
2,Copperas Cove,US,77.16,5.75,1015
3,Copperas Cove,US,76.84,6.91,1015
4,Copperas Cove,US,80.37,0.00,1014
5,Uvalde,US,79.21,3.44,1011
6,Uvalde,US,79.47,11.50,1014
7,Uvalde,US,77.70,3.44,1015
8,Uvalde,US,78.71,9.22,1015
9,Uvalde,US,79.21,6.91,1015


In [20]:
df[df["Country"]=="US"]

,City Name,Country,Temperature,Wind_Speed,Pressure
0,Leander,US,78.82,1.99,1005
1,Leander,US,79.05,1.99,1008
2,Leander,US,74.95,1.01,1010
3,Leander,US,77.36,1.99,1010
4,Leander,US,78.57,0.00,1010
5,Port Arthur,US,78.60,4.61,1008
6,Port Arthur,US,78.60,10.36,1009
7,Port Arthur,US,76.21,5.75,1014
8,Port Arthur,US,77.77,4.61,1014
9,Port Arthur,US,79.34,11.50,1014
